In [329]:
## Import and read the raw data
import pandas as pd

data = pd.read_csv('bank-additional-full.csv', delimiter=';')
data.head()

# Description of fields
# age               numeric
# job               categorical
# marital           categorical
# education         categorical
# default           categorical
# housing           categorical
# loan              categorical
# contact           categorical
# month             categorical
# day               categorical
# duration          numeric
# campaign          numeric
# pdays             numeric
# previous          numeric
# poutcome          categorical
# emp.var.rate      numeric
# cons.price.idx    numeric
# cons.conf.idx     numeric
# euribor3m         numeric
# nr.employed       numeric

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [330]:
import numpy as np
# from numpy import nan

data['contacted_pdays'] = data['pdays']

data.loc[data.contacted_pdays == 999, 'contacted_pdays'] = 0
data.loc[data.contacted_pdays != 0, 'contacted_pdays'] = 1

# Get values from pdays column,remove 999 values
values = np.array(data.pdays)
values = values[values != 999]
median = np.median(values)

# data.loc[data.pdays == 999, 'pdays'] = 999



columns = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'contacted_pdays', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']

data = data.reindex(columns=columns)

In [331]:
import numpy as np
# Encode the desired target ("y") to a binary representation
from sklearn.preprocessing import OrdinalEncoder

def encode_to_ordinal(array):
    ordinal_encoder = OrdinalEncoder(dtype=np.int)
    return ordinal_encoder.fit_transform(array.reshape(-1,1))

ordinal_encoder = OrdinalEncoder()
y_encoded = encode_to_ordinal(data.y.values)


In [332]:
from sklearn.preprocessing import OneHotEncoder

# One-hot encoding for categorical features
one_hot_encoder = OneHotEncoder(dtype=np.int, sparse=True)

# start assembling cleaned data
clean_data = pd.DataFrame(one_hot_encoder.fit_transform(data[['job', 'marital','education','default','housing','loan','contact','month','day_of_week','poutcome']]).toarray())



In [333]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Perform feature scaling on numerical values
clean_data['age'] = scaler.fit_transform(data[['age']])
clean_data['campaign'] = scaler.fit_transform(data[['campaign']])
clean_data['contacted_pdays'] = data[['contacted_pdays']]
clean_data['pdays'] = standard_scaler.fit_transform(data[['pdays']])
clean_data['previous'] = scaler.fit_transform(data[['previous']])
clean_data['emp.var.rate'] = scaler.fit_transform(data[['emp.var.rate']])
clean_data['cons.price.idx'] = scaler.fit_transform(data[['cons.price.idx']])
clean_data['cons.conf.idx'] = scaler.fit_transform(data[['cons.conf.idx']])
clean_data['euribor3m'] = scaler.fit_transform(data[['euribor3m']])
clean_data['nr.employed'] = scaler.fit_transform(data[['nr.employed']])
clean_data['y'] = y_encoded

In [334]:
## Generate histograms for numerical features in processed data
# import matplotlib.pyplot as plt

# %matplotlib inline

# histogram_columns = clean_data[['age', 'campaign', 'contacted_pdays', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y']]
# histogram_columns.hist(bins=50, figsize=(20,15))

In [335]:
## Search for correlations
# correlation_matrix = clean_data.corr()
# correlation_matrix['y'].sort_values(ascending = False)

In [336]:
# Split training and test sets
from sklearn.model_selection import train_test_split

X = clean_data.loc[:, clean_data.columns != 'y']
y = clean_data['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)


In [337]:
import tensorflow as tf

# Build and compile TF model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(63, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])


In [338]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
863/863 [==============================] - 1s 1ms/step - loss: 0.3291 - accuracy: 0.8713
Epoch 2/10
863/863 [==============================] - 1s 1ms/step - loss: 0.2738 - accuracy: 0.9005
Epoch 3/10
863/863 [==============================] - 1s 2ms/step - loss: 0.2744 - accuracy: 0.9002
Epoch 4/10
863/863 [==============================] - 1s 2ms/step - loss: 0.2707 - accuracy: 0.9010
Epoch 5/10
863/863 [==============================] - 1s 1ms/step - loss: 0.2729 - accuracy: 0.9010
Epoch 6/10
863/863 [==============================] - 1s 2ms/step - loss: 0.2693 - accuracy: 0.9018
Epoch 7/10
863/863 [==============================] - 1s 1ms/step - loss: 0.2658 - accuracy: 0.9050
Epoch 8/10
863/863 [==============================] - 1s 1ms/step - loss: 0.2623 - accuracy: 0.9054
Epoch 9/10
863/863 [==============================] - 1s 1ms/step - loss: 0.2623 - accuracy: 0.9056
Epoch 10/10
863/863 [==============================] - 1s 1ms/step - loss: 0.2593 - accuracy: 0.9071